## Virulance HMM Table Analysis

In [8]:
from Bio import SearchIO
import pandas as pd
num_hit = 0
#now parse the output
hit_list = []
hit_ids = []
with open('hmm_table.txt', 'r') as input:
    for qresult in SearchIO.parse(input, 'hmmer3-text'):
        query_id = qresult.id  #sequence ID from fasta
        query_len = qresult.seq_len
        hits = qresult.hits
        hit_ids = hit_ids+qresult.hit_keys
        hit_list.append(hits)
        num_hits = len(hits)
        num_hit = num_hit + num_hits
print(num_hit)

19786


### Exploring Hit Table

In [9]:
len(hit_ids)

19786

In [2]:
#df = pd.read_table("hmm_table_test.txt")

In [10]:
uniq_hits = list(set(hit_ids))
len(uniq_hits)

15389

### Count of Viral Species in Uniprot Database

In [11]:
from Bio import SeqIO
ids = []
names = []
for seq_record in SeqIO.parse("viral.1.protein.faa", "fasta"):
    ids.append(seq_record.id)
    names.append(seq_record.description)

In [83]:
len(names)

252202

In [12]:
viral_names = [x.split('[', 1)[1].split(']')[0] for x in names]

In [195]:
len(viral_names)
viral_names[1:10]

['Paenibacillus phage phiIBB_Pl23',
 'Paenibacillus phage phiIBB_Pl23',
 'Paenibacillus phage phiIBB_Pl23',
 'Paenibacillus phage phiIBB_Pl23',
 'Paenibacillus phage phiIBB_Pl23',
 'Paenibacillus phage phiIBB_Pl23',
 'Paenibacillus phage phiIBB_Pl23',
 'Paenibacillus phage phiIBB_Pl23',
 'Paenibacillus phage phiIBB_Pl23']

In [122]:
from collections import Counter

freq = Counter(viral_names)
freq

Counter({'Paenibacillus phage phiIBB_Pl23': 68,
         'Bacillus virus Curly': 77,
         'Shigella phage SfII': 58,
         'Salmonella phage FSL SP-058': 86,
         'Salmonella phage FSL SP-076': 82,
         'Human endogenous retrovirus K113': 1,
         'Tomato mottle mosaic virus': 4,
         'Porcine stool-associated circular virus 2': 2,
         'Porcine stool-associated circular virus 3': 2,
         'Sweet potato latent virus': 1,
         'Bacillus phage B4': 277,
         'Pseudomonas phage MPK7': 53,
         'Drosophila obscura sigmavirus 10A': 6,
         'Eel virus European X': 6,
         'Lactobacillus phage phiAQ113': 56,
         'Feline astrovirus 2': 3,
         'Euphorbia mosaic virus': 2,
         'Hemileuca sp. nucleopolyhedrovirus': 137,
         'Staphylococcus phage SA12': 58,
         'Pokeweed mosaic virus': 11,
         'Sulfolobus virus STSV2': 75,
         'Red clover cryptic virus 1': 2,
         'Rhizoctonia cerealis alphaendornavirus 1': 1,


In [121]:
#number of unique viral 
virnum = list(set(viral_names))
len(virnum)

6580

In [13]:
df = pd.DataFrame({'IDs': ids, 
                   'Viral Names': viral_names, 
                   'CompleteRecord': names}, columns=['IDs','Viral Names', 'Complete Record'])

In [107]:
df.head()

,IDs,Viral Names,Complete Record
0,YP_008320337.1,Paenibacillus phage phiIBB_Pl23,YP_008320337.1 terminase small subunit [Paenib...
1,YP_008320338.1,Paenibacillus phage phiIBB_Pl23,YP_008320338.1 terminase large subunit [Paenib...
2,YP_008320339.1,Paenibacillus phage phiIBB_Pl23,YP_008320339.1 portal protein [Paenibacillus p...
3,YP_008320340.1,Paenibacillus phage phiIBB_Pl23,YP_008320340.1 Clp protease-like protein [Paen...
4,YP_008320341.1,Paenibacillus phage phiIBB_Pl23,YP_008320341.1 major capsid protein [Paenibaci...


### Add Hit by Protein ID 

In [14]:
df['Virulence'] = [1 if x in uniq_hits else 0 for x in df['IDs']]

In [128]:
import pickle
pickle.dump(df, open('Virulance_Dataframe.p', 'wb'))

In [149]:
## Group by
test2 = df.groupby('Viral Names').sum()
sum(test2.Virulence)

15389

### Filter Viral Names by Phages

In [15]:
y = df['Viral Names'].str.contains('phage')

In [16]:
df2=df[y]

In [146]:
## 155606/252202 = 62.70%
len(df2)

155606

In [17]:
## Group by
test = df2.groupby('Viral Names').sum()

In [228]:
len(test)

1510

In [148]:
sum(test.Virulence)

9816

In [ ]:
import matplotlib.pyplot as plt
import matplotlib as mpl
%matplotlib inline
items = test.Virulence
plt.xkcd()
plt.boxplot(items)

In [12]:
zeros_in_test = test[test.Virulence == 0]
low_count = test[(test.Virulence >= 1) & (test.Virulence <= 10)]
tens_count = test[(test.Virulence > 10) & (test.Virulence <= 100)]

In [15]:
len(zeros_in_test)

84

In [13]:
len(low_count)

1147

In [14]:
len(tens_count)

279

In [18]:
## Group by Bacterial Genus
genus_name = test.index
demo2 = [x.split(' ',1)[0] for x in genus_name]

In [196]:
len(demo2)

1510

In [19]:
test['Genus'] = demo2

In [20]:
Virulence_Avg = test.groupby('Genus').mean()
Virulence_Avg = Virulence_Avg.sort_values(by=['Virulence'], ascending=False)
Virulence_Avg.columns = ['Avg Number of Virulent Matches']
Virulence_Avg.head()

,Avg Number of Virulent Matches
Genus,
Aureococcus,41.000000
Brevibacillus,18.833333
Prochlorococcus,14.666667
Bacillus,14.414634
Dickeya,14.000000


In [21]:
Number_of_phages = test.groupby('Genus').count()
Number_of_phages = Number_of_phages.sort_values(by=['Virulence'], ascending=False)
Number_of_phages.columns = ['Number of Phages in Genus']
Number_of_phages.head()

,Number of Phages in Genus
Genus,
Mycobacterium,249
Pseudomonas,112
Escherichia,92
Bacillus,82
Salmonella,67


In [22]:
### Number of Viruses over 30 in Genus
x = Number_of_phages[Number_of_phages['Number of Phages in Genus']>=30]
len(x)

13

In [23]:
abundant_phages = x.index

In [24]:
Abundant_df = Virulence_Avg[(Virulence_Avg.index).isin(abundant_phages)]
Abundant_df

,Avg Number of Virulent Matches
Genus,
Bacillus,14.414634
Staphylococcus,9.965517
Lactobacillus,8.424242
Escherichia,7.543478
Streptococcus,6.953488
Salmonella,6.507463
Enterobacteria,5.677966
Mycobacterium,5.542169
Vibrio,5.272727


In [98]:
from scipy import stats
## Bacilus Distribution
bacillus_disto = test[test['Genus']=='Bacillus']
l = list(bacillus_disto['Virulence'])
q = stats.shapiro(l)
print("Shaprio-Wilks Test of Normality of Bacillus Virulent Genes p-value = %s" % q[1])
## Staphylococcus Distribution
Staphylococcus_disto = test[test['Genus']=='Staphylococcus']
y = list(Staphylococcus_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Staphylococcus Virulent Genes p-value = %s" % q[1])
## Lactobacillus Distribution
Lactobacillus_disto = test[test['Genus']=='Lactobacillus']
y = list(Lactobacillus_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Lactobacillus Virulent Genes p-value = %s" % q[1])
## Escherichia Distribution
Escherichia_disto = test[test['Genus']=='Escherichia']
y = list(Escherichia_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Escherichia Virulent Genes p-value = %s" % q[1])
## Streptococcus Distribution
Streptococcus_disto = test[test['Genus']=='Streptococcus']
y = list(Streptococcus_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Streptococcus Virulent Genes p-value = %s" % q[1])
## Salmonella Distribution
Salmonella_disto = test[test['Genus']=='Salmonella']
y = list(Salmonella_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Salmonella Virulent Genes p-value = %s" % q[1])
## Enterobacteria Distribution
Enterobacteria_disto = test[test['Genus']=='Enterobacteria']
y = list(Enterobacteria_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Enterobacteria Virulent Genes p-value = %s" % q[1])
## Mycobacterium Distribution
Mycobacterium_disto = test[test['Genus']=='Mycobacterium']
y = list(Mycobacterium_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Mycobacterium Virulent Genes p-value = %s" % q[1])
## Vibrio Distribution
Vibrio_disto = test[test['Genus']=='Vibrio']
y = list(Vibrio_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Vibrio Virulent Genes p-value = %s" % q[1])
## Gordonia Distribution
Gordonia_disto = test[test['Genus']=='Gordonia']
y = list(Gordonia_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Gordonia Virulent Genes p-value = %s" % q[1])
## Lactococcus Distribution
Lactococcus_disto = test[test['Genus']=='Lactococcus']
y = list(Lactococcus_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Lactococcus Virulent Genes p-value = %s" % q[1])
## Pseudomonas Distribution
Pseudomonas_disto = test[test['Genus']=='Pseudomonas']
t = list(Pseudomonas_disto['Virulence'])
q = stats.shapiro(t)
print("Shaprio-Wilks Test of Normality of Pseudomonas Virulent Genes p-value = %s" % q[1])
## Propionibacterium Distribution
Propionibacterium_disto = test[test['Genus']=='Propionibacterium']
y = list(Propionibacterium_disto['Virulence'])
q = stats.shapiro(y)
print("Shaprio-Wilks Test of Normality of Propionibacterium Virulent Genes p-value = %s" % q[1])

Shaprio-Wilks Test of Normality of Bacillus Virulent Genes p-value = 0.0100819077343
Shaprio-Wilks Test of Normality of Staphylococcus Virulent Genes p-value = 0.000621999613941
Shaprio-Wilks Test of Normality of Lactobacillus Virulent Genes p-value = 0.168927758932
Shaprio-Wilks Test of Normality of Escherichia Virulent Genes p-value = 4.39726392187e-07
Shaprio-Wilks Test of Normality of Streptococcus Virulent Genes p-value = 0.529277861118
Shaprio-Wilks Test of Normality of Salmonella Virulent Genes p-value = 0.0118698384613
Shaprio-Wilks Test of Normality of Enterobacteria Virulent Genes p-value = 9.23852257984e-06
Shaprio-Wilks Test of Normality of Mycobacterium Virulent Genes p-value = 1.5640644051e-05
Shaprio-Wilks Test of Normality of Vibrio Virulent Genes p-value = 5.77306855121e-06
Shaprio-Wilks Test of Normality of Gordonia Virulent Genes p-value = 0.0050412658602
Shaprio-Wilks Test of Normality of Lactococcus Virulent Genes p-value = 0.00584692088887
Shaprio-Wilks Test of No

### Stats To-Do
Considering Normlizing counts between 0 and 1 then applying the same normalization to observation. Then performing a one sample t-test or outlier detection

In [97]:
import numpy as np
z = np.mean(y)
variance = np.var(y)
variance

1.0612730517549076

In [102]:
import numpy as np
z = np.mean(l)
variance = np.var(l)
variance
#z

43.998810232004764

In [78]:
qw = stats.ttest_1samp(y,3)
qw

Ttest_1sampResult(statistic=-2.2460728058830486, pvalue=0.030290382881324712)

In [88]:
# Calculate Sample Mean
def mean(x):
    return sum(x)/(len(x)*1.0) 

# Calculate Sample Standard Deviation
def sd(x):
    xb = sum(x)/(len(x)*1.0) 
    return (sum([(i-xb)**2 for i in x])/(len(x)-1))**(0.5) 

In [91]:
def one_sample_ttest(x,mu,alt='neq'):
    # Calculate t statistic and p-value.
    ttest_onesample = stats.ttest_1samp(x, mu)
    # Extract t-stat
    t = ttest_onesample[0]
    # Sample Size
    n = len(x)
    # Degress of Freedom
    df = n-1
    # Calculate Standard Error
    se = sd(x)/((n)**(0.5))
    # Calculate the correct p value if gt is selected
    if(alt == 'gt'):   
        # Correct p-value for one sided alt
        if(t>=0):
            p = (ttest_onesample[1])/(2.0)
        elif(t==0):
            p = 1-(ttest_onesample[1])/(2.0)
        if(t<=0):
            p = (ttest_onesample[1])/(2.0)
            
        hypothesis = "less than"
        # Calculate 95 percent conf limits
        ts = stats.t.ppf(.05, df)
        ul = mean(x)-se*ts
        ll = float("-inf")
    else:
        p = ttest_onesample[1]
        # Calculate 95 percent conf limits
        hypothesis = "not equal to"
        ts = stats.t.ppf(0.025, df)
        ll = mean(x)-se*ts
        ul = mean(x)+se*ts
        
    print "One Sample t-test:"
    print "t = %.4f," % t,"df = %.f," % df, "p-value = %.4f" % p 
    print "alternative hypothesis: true mean is", hypothesis , mu, "\n"
    print "95 percent confidence interval:"
    print "\t (%.4f," % ll , "%.4f)" % ul, "\n"

In [96]:
one_sample_ttest(y, 3, 'gt')

One Sample t-test:
t = -2.2461, df = 40, p-value = 0.0151
alternative hypothesis: true mean is less than 3 

95 percent confidence interval:
	 (-inf, 2.9084) 

